# Proper code

## Shapley function

In [1]:
import openturns as ot
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from itertools import permutations
from shapley.shapley import shapley_indices
from shapley.plots import set_style_paper, plot_violin

%matplotlib inline

%load_ext autoreload
%autoreload 2

set_style_paper()

savefigs = True

## Function for conditional gaussian vector

### Calcul of conditional mean and variance

In [2]:
def condMVN(mean, cov, dependent_ind, given_ind, X_given):
    """ Returns conditional mean and variance of X[dependent.ind] | X[given.ind] = X.given
    where X is multivariateNormal(mean = mean, covariance = cov)"""
    
    cov = np.array(cov)
    
    B = cov.take(dependent_ind, axis = 1)
    B = B[dependent_ind]
    
    C = cov.take(dependent_ind, axis = 1)
    C = C[given_ind]
    
    D = cov.take(given_ind, axis = 1)
    D = D[given_ind]
    
    CDinv = np.dot(np.transpose(C),np.linalg.inv(D))
    
    condMean = mean[dependent_ind] + np.dot(CDinv,(X_given - mean[given_ind]))
    condVar = B - np.dot(CDinv,C)
    condVar = ot.CovarianceMatrix(condVar)
    
    return condMean,condVar

### Generate conditional law

In [3]:
def r_condMVN(n, mean, cov, dependent_ind, given_ind, X_given):
    """ Function to simulate conditional gaussian distribution of X[dependent.ind] | X[given.ind] = X.given
    where X is multivariateNormal(mean = mean, covariance = cov)"""
    
    cond_mean,cond_var = condMVN(mean, cov, dependent_ind, given_ind, X_given)
    distribution = ot.Normal(cond_mean,cond_var)
    return distribution.getSample(n)

## Evaluation Shapley effects on linear gaussian model

### Ni = 3

In [4]:
def gaussian_model(X):
    return np.sum(X,1)

d = 3
moyenne = np.zeros(3)
cov = np.array([[1.0, 0, 0], [0, 1.0, 1.8], [0, 1.8, 4.0]])
cov = ot.CovarianceMatrix(cov)

def Xall(n):
    distribution = ot.Normal(moyenne,cov)
    return distribution.getSample(n)

def Xcond(n, Sj, Sjc, xjc):
    if Sjc is None:
        cov_int = np.array(cov)
        cov_int = cov_int.take(Sj, axis = 1)
        cov_int = cov_int[Sj, :]        
        cov_int = ot.CovarianceMatrix(cov_int)
        distribution = ot.Normal(moyenne[Sj],cov_int)
        return distribution.getSample(n)
    else:
        return r_condMVN(n,mean = moyenne, cov = cov, dependent_ind = Sj, given_ind = Sjc, X_given = xjc)

In [5]:
# Exact method
method = 'exact'
m = None
Nv = 10**4
No = 10**3
Ni = 3

index = shapley_indices(method,m,gaussian_model, Xall, Xcond, d, Nv, No, Ni)
print('Exact method \n' + str(index) + '\n\n')

Exact method 
                         S1        S2        S3
Shapley effects    0.103209  0.423642  0.473149
First order Sobol  0.099529  0.811843  0.870678
Total Sobol        0.102771  0.020522  0.079368




In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
index.T.plot(kind='bar', ax=ax)
ax.set_ylabel('Indice values')
ax.set_title("Additive Gaussian - Shapley effects with exact method - $Nv=%d$, $No=%d$, $Ni=%d$" % (Nv, No, Ni))

fig.tight_layout()
if savefigs:
    fig.savefig('./output/gaussian_shapley_exact.pdf')

In [ ]:
# Random method
method = 'random'
m = 6000
Nv = 10**4
No = 1
Ni = 3

index = shapley_indices(method,m,gaussian_model, Xall, Xcond, d, Nv, No, Ni)
print('Random method \n' + str(index) + '\n')

### Ni = 100

In [ ]:
def gaussian_model(X):
    return np.sum(X,1)

d = 3
moyenne = np.zeros(3)
cov = np.array([[1.0, 0, 0], [0, 1.0, 1.8], [0, 1.8, 4.0]])
cov = ot.CovarianceMatrix(cov)

def Xall(n):
    distribution = ot.Normal(moyenne,cov)
    return distribution.getSample(n)

def Xcond(n, Sj, Sjc, xjc):
    if Sjc is None:
        cov_int = np.array(cov)
        cov_int = cov_int.take(Sj, axis = 1)
        cov_int = cov_int[Sj]        
        cov_int = ot.CovarianceMatrix(cov_int)
        distribution = ot.Normal(moyenne[Sj],cov_int)
        return distribution.getSample(n)
    else:
        return r_condMVN(n,mean = moyenne, cov = cov, dependent_ind = Sj, given_ind = Sjc, X_given = xjc)

# Exact method
method = 'exact'
m = None
Nv = 10**4
No = 10**3
Ni = 100

index = shapley_indices(method,m,gaussian_model, Xall, Xcond, d, Nv, No, Ni)
print('Exact method \n' + str(index) + '\n\n')

# Random method
method = 'random'
m = 6000
Nv = 10**4
No = 1
Ni = 100

index = shapley_indices(method,m,gaussian_model, Xall, Xcond, d, Nv, No, Ni)
print('Random method \n' + str(index) + '\n')